# Itility - Analyst Testcase - Serverdata v2.0

## Importing the data
First we import the data using Pandas function to read Excel files.

In [1]:
import pandas as pd
import numpy as np

server_data = pd.DataFrame()
server_data = pd.read_excel('Analyst Testcase_Serverdata v2.0 020415.xlsx', 
                            'Server Data',
                            index_col = None,
                            na_values = ['NA'])
print server_data.head()

   Timestamp    Servername                Metric Unit      Average        Max  \
0 2014-09-22  ITLT_C1_S001    mem.active.average   KB  190856.3838  611107.65   
1 2014-09-22  ITLT_C1_S001  cpu.usagemhz.average  MHz      46.8072     303.60   
2 2014-09-23  ITLT_C1_S001    mem.active.average   KB  179302.9824  661132.24   
3 2014-09-23  ITLT_C1_S001  cpu.usagemhz.average  MHz      50.0955     300.84   
4 2014-09-24  ITLT_C1_S001    mem.active.average   KB  208730.8190  786261.83   

        Min  MemoryMB  NumCpu  CpuMHz  Cluster  
0  41101.20      4096       2    5320        1  
1     25.25      4096       2    5320        1  
2  62408.58      4096       2    5320        1  
3     23.04      4096       2    5320        1  
4  54718.75      4096       2    5320        1  


## Slicing the data
We need to slice the DataFrame, since it contains memory and CPU data on different rows.

### Retrieve the memory data

In [2]:
mem_data = pd.DataFrame()
mem_data = server_data.loc[server_data['Metric'] == 'mem.active.average']
print mem_data.describe()

            Average           Max           Min      MemoryMB        NumCpu  \
count  5.814300e+04  5.814300e+04  5.814300e+04  58143.000000  58143.000000   
mean   8.660496e+05  3.573797e+06  3.795001e+05  11611.071840      2.721807   
std    8.104414e+05  3.861459e+07  5.065735e+05   8777.473166      1.400067   
min    0.000000e+00  0.000000e+00  0.000000e+00   2048.000000      1.000000   
25%    2.890374e+05  9.966694e+05  2.879983e+04   4096.000000      2.000000   
50%    6.888673e+05  2.231536e+06  2.214585e+05   8192.000000      2.000000   
75%    1.148850e+06  3.835241e+06  5.480531e+05  16384.000000      4.000000   
max    1.111794e+07  9.271316e+09  1.139870e+07  40960.000000      8.000000   

             CpuMHz       Cluster  
count  58143.000000  58143.000000  
mean    6400.469188      3.382196  
std     3347.338371      1.286721  
min     2000.000000      1.000000  
25%     4800.000000      3.000000  
50%     4800.000000      3.000000  
75%     9600.000000      4.000000  


### Retrieve the CPU data

In [3]:
cpu_data = pd.DataFrame()
cpu_data = server_data.loc[server_data['Metric'] == 'cpu.usagemhz.average']
print cpu_data.describe()

           Average           Max           Min      MemoryMB        NumCpu  \
count  58143.00000  58143.000000  58143.000000  58143.000000  58143.000000   
mean     460.93169   2038.818646    254.759717  11611.071840      2.721807   
std     1289.17880   3748.750214    774.244759   8777.473166      1.400067   
min        1.62900      1.840000      0.900000   2048.000000      1.000000   
25%       54.75540    315.605000     26.160000   4096.000000      2.000000   
50%      183.04000   1058.270000     90.300000   8192.000000      2.000000   
75%      413.23005   2198.700000    237.875000  16384.000000      4.000000   
max    48931.42960  67736.960000  50418.400000  40960.000000      8.000000   

             CpuMHz       Cluster  
count  58143.000000  58143.000000  
mean    6400.469188      3.382196  
std     3347.338371      1.286721  
min     2000.000000      1.000000  
25%     4800.000000      3.000000  
50%     4800.000000      3.000000  
75%     9600.000000      4.000000  
max    19

### Merge the rows for CPU and memory to one row

In [4]:
cpu_cols = ['Timestamp', 'Servername', 'Average', 'Max', 'Min', 'MemoryMB', 'NumCpu', 'CpuMHz', 'Cluster']
mem_cols = ['Timestamp', 'Servername', 'Average', 'Max', 'Min']
server_data = pd.merge(
                        cpu_data[cpu_cols],
                        mem_data[mem_cols],
                        on=['Timestamp', 'Servername'],
                        how='inner',
                        suffixes=('_cpu', '_mem')
                    )

### Show the columns of the data

In [35]:
print server_data.head()

   Timestamp    Servername  Average_cpu  Max_cpu  Min_cpu  MemoryMB  NumCpu  \
0 2014-09-22  ITLT_C1_S001      46.8072   303.60    25.25      4096       2   
1 2014-09-23  ITLT_C1_S001      50.0955   300.84    23.04      4096       2   
2 2014-09-24  ITLT_C1_S001      45.6967   350.86    26.16      4096       2   
3 2014-09-25  ITLT_C1_S001      50.6412   339.36    22.50      4096       2   
4 2014-09-26  ITLT_C1_S001      47.6805   455.40    25.30      4096       2   

   CpuMHz  Cluster  Average_mem    Max_mem   Min_mem  
0    5320        1  190856.3838  611107.65  41101.20  
1    5320        1  179302.9824  661132.24  62408.58  
2    5320        1  208730.8190  786261.83  54718.75  
3    5320        1  180816.3721  894138.30  46805.04  
4    5320        1  190580.0035  910945.49  42778.80  


### Example of the data

In [34]:
print server_data[server_data['Servername'] == 'ITLT_C1_S001'].describe()

       Average_cpu     Max_cpu    Min_cpu  MemoryMB  NumCpu  CpuMHz  Cluster  \
count    61.000000   61.000000  61.000000      61.0    61.0    61.0     61.0   
mean     47.077380  380.306393  22.219836    4096.0     2.0  5320.0      1.0   
std       6.212575  153.489274   4.650210       0.0     0.0     0.0      0.0   
min      23.040000   24.480000   0.920000    4096.0     2.0  5320.0      1.0   
25%      44.754500  310.500000  20.930000    4096.0     2.0  5320.0      1.0   
50%      47.960000  355.350000  22.440000    4096.0     2.0  5320.0      1.0   
75%      50.440000  425.320000  24.610000    4096.0     2.0  5320.0      1.0   
max      64.745700  780.000000  30.000000    4096.0     2.0  5320.0      1.0   

         Average_mem       Max_mem        Min_mem  
count      61.000000  6.100000e+01      61.000000  
mean   194676.097474  9.326388e+05   53334.612787  
std     19814.837173  5.175791e+05   42398.491987  
min    155840.106400  1.808000e+05    8639.640000  
25%    180263.31000

In [33]:
for label, df in server_data.groupby('Servername'):
    if df['MemoryMB'].std() != 0:
        print 'Memory increase', label, df['MemoryMB'].std()
    if df['CpuMHz'].std() > 1000:
        print 'CPU increase', label, df['CpuMHz'].std()

Memory increase ITLT_C2_S134 6157.17043312
CPU increase ITLT_C2_S134 1403.00107526
CPU increase ITLT_C3_X132 1168.73983977
Memory increase ITLT_C3_X135 1466.80079516
Memory increase ITLT_C3_X428 1032.94340124
Memory increase ITLT_C3_X495 1320.54272283
Memory increase ITLT_C3_X496 1324.28848134
Memory increase ITLT_C3_X498 2322.21629576
Memory increase ITLT_C3_X499 2340.0426817
Memory increase ITLT_C3_X500 1161.10814788
Memory increase ITLT_C3_X501 1161.10814788
CPU increase ITLT_C4_X573 1174.68766363
CPU increase ITLT_C4_X574 1180.55549635
CPU increase ITLT_C4_X575 1180.55549635
Memory increase ITLT_C4_X578 1032.64150342
CPU increase ITLT_C4_X580 1180.55549635
CPU increase ITLT_C6_S030 1199.32184227
CPU increase ITLT_C6_S031 1199.32184227
Memory increase ITLT_C6_X878 967.61433433
CPU increase ITLT_C6_X881 3669.45977835
CPU increase ITLT_C6_X882 3669.45977835
Memory increase ITLT_C6_X884 9434.22189305
CPU increase ITLT_C6_X884 1223.15325945
CPU increase ITLT_C6_X885 1223.15325945


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plot.plot(server_data['Timestamp'], '')

In [14]:
%matplotlib inline
import matplotlib.pyplot as plt

for key, grp in server_data.groupby(['Servername', 'Timestamp']):
#     plt.plot(grp['Average_cpu'], label=key)
#     grp['D'] = pd.rolling_mean(grp['B'], window=5)    
#     plt.plot(grp['D'], label='rolling ({k})'.format(k=key))
# plt.legend(loc='best')    
plt.show()
plt.xticks([],[server_data['Timestamp']])

KeyboardInterrupt: 